In [8]:
import os
import pyspark
import logging
from pyspark.ml.regression import LinearRegression

In [2]:
logging.basicConfig(level=logging.INFO)
os.chdir('../src/')
os.getcwd()

'/home/jerome/misc/projects/programming/forest-for-flood/src'

In [3]:
from d01_data.read_parquet import clean_parquet_df
from d02_intermediate.to_parquet import clean_write_parquet
from d03_processing.input_model import transform_into_vector
from d04_modelling.train_model import fit_model
from d05_model_evaluation.metrics import create_predictions, calculate_metrics
os.chdir('../')
os.getcwd()

'/home/jerome/misc/projects/programming/forest-for-flood'

In [4]:
spark = (
    pyspark.sql.SparkSession.builder.master("local[*]")
    .config("spark.driver.memory", "10g")
    .getOrCreate()
)
sc = spark.sparkContext

sc.setLogLevel("FATAL")

23/05/09 16:38:40 WARN Utils: Your hostname, Starscream resolves to a loopback address: 127.0.1.1; using 192.168.10.26 instead (on interface wlan0)
23/05/09 16:38:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/09 16:38:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
df = clean_parquet_df(spark)
train_df, test_df, validation_df = transform_into_vector(spark, df)

In [19]:
model = LinearRegression(labelCol="totalinsurancepremiumofthepolicy", maxIter=5000)

In [20]:
trained_model = model.fit(train_df)

In [21]:
predictions_df = trained_model.transform(test_df)

In [22]:
metrics_dict = calculate_metrics(predictions_df)

/home/jerome/misc/projects/programming/basic-venv/lib/python3.11/site-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [23]:
metrics_dict

{'r2': -5.106408626084753,
 'explainedVariance': 3590781.9537764806,
 'meanAbsoluteError': 681.8144323289797,
 'meanSquaredError': 3081189.43383709,
 'rootMeanSquaredError': 1755.3317161827533}